In [ ]:
!pip install Unidecode
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import pandas as pd
import json
import glob
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools
import concurrent.futures
from bs4 import BeautifulSoup
from unidecode import unidecode

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Datos/CORFO"
results_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Resultados/CORFO"

In [ ]:
df_corfo = pd.read_excel(os.path.join(results_dir, "data_corfo_ruts_imputados.xlsx"))
df_corfo

,Unnamed: 0,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley,rut_normalizado,rut_valido,nombre_normalizado
0,0,1,2023.0,23CH-246113,Capital Humano,BENEFICIARIO / DESARROLLADOR,Nuevas estrategias de microencapsulación para ...,77131055-9,Liva Company SPA,PERSONA JURIDICA COMERCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,30000000.0,3333334.0,3333334.0,NaN,77131055-9,True,LIVA COMPANY SPA
1,1,2,2023.0,23CH-246116,Capital Humano,BENEFICIARIO / DESARROLLADOR,Aprender haciendo bakeries de bajo índice glic...,77208333-5,ELAB. Y COM. CASONA EL MONTE LTDA,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,29985000.0,7059200.0,4273200.0,NaN,77208333-5,True,ELAB Y COM CASONA EL MONTE LTDA
2,2,3,2023.0,23CH-246118,Capital Humano,BENEFICIARIO / DESARROLLADOR,Optimización de medios de cultivo sustentables...,77148405-0,LUYEF BIOTECHNOLOGIES CHILE SPA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3340000.0,3340000.0,NaN,77148405-0,True,LUYEF BIOTECHNOLOGIES CHILE SPA
3,3,4,2023.0,23CH-246123,Capital Humano,BENEFICIARIO / DESARROLLADOR,Producción de proteínas recombinantes a partir...,77065948-5,Pewman Innovation spa,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",28500000.0,7800000.0,5400000.0,NaN,77065948-5,True,PEWMAN INNOVATION SPA
4,4,5,2023.0,23CH-246125,Capital Humano,BENEFICIARIO / DESARROLLADOR,NGB-eDNA. Plataforma genética no-invasiva para...,77063937-9,Ecología y Genética Ambiental SpA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3500000.0,3500000.0,NaN,77063937-9,True,ECOLOGIA Y GENETICA AMBIENTAL SPA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10566,10566,1344,NaN,PI-998,Ley I+D,mandante,Eficacia y toxicidad del uso de una familia de...,77424780-7,EWOS CHILE ALIMENTOS LIMITADA,PERSONA JURIDICA COMERCIAL,D - INDUSTRIAS MANUFACTURERAS NO METALICAS,NaN,NaN,NaN,76835703.0,77424780-7,True,EWOS CHILE ALIMENTOS LIMITADA
10567,10567,1345,NaN,PI-99869,Ley I+D,mandante,Análisis Proteómico por Machine-Learning: Perf...,76221912-3,LABORATORIO PATHOVET LIMITADA,PERSONA JURIDICA COMERCIAL,O - SERVICIOS SOCIALES Y DE SALUD,NaN,NaN,NaN,23076375.0,76221912-3,True,LABORATORIO PATHOVET LIMITADA
10568,10568,1346,NaN,PI-99871,Ley I+D,mandante,Desarrollo de un proceso de recubrimiento o lí...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19770000.0,76082953-6,True,NANOTEC SA
10569,10569,1347,NaN,PI-99872,Ley I+D,mandante,Nuevos usos de nanopartículas metálicas: Desar...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19800000.0,76082953-6,True,NANOTEC SA


In [ ]:
df_corfo["rut_sin_dv"] = df_corfo["rut_normalizado"]
df_corfo.loc[df_corfo["rut_valido"], "rut_sin_dv"] = df_corfo.loc[df_corfo["rut_valido"], "rut_normalizado"].str.split("-").str[0]
df_corfo

,Unnamed: 0,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley,rut_normalizado,rut_valido,nombre_normalizado,rut_sin_dv
0,0,1,2023.0,23CH-246113,Capital Humano,BENEFICIARIO / DESARROLLADOR,Nuevas estrategias de microencapsulación para ...,77131055-9,Liva Company SPA,PERSONA JURIDICA COMERCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,30000000.0,3333334.0,3333334.0,NaN,77131055-9,True,LIVA COMPANY SPA,77131055
1,1,2,2023.0,23CH-246116,Capital Humano,BENEFICIARIO / DESARROLLADOR,Aprender haciendo bakeries de bajo índice glic...,77208333-5,ELAB. Y COM. CASONA EL MONTE LTDA,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,29985000.0,7059200.0,4273200.0,NaN,77208333-5,True,ELAB Y COM CASONA EL MONTE LTDA,77208333
2,2,3,2023.0,23CH-246118,Capital Humano,BENEFICIARIO / DESARROLLADOR,Optimización de medios de cultivo sustentables...,77148405-0,LUYEF BIOTECHNOLOGIES CHILE SPA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3340000.0,3340000.0,NaN,77148405-0,True,LUYEF BIOTECHNOLOGIES CHILE SPA,77148405
3,3,4,2023.0,23CH-246123,Capital Humano,BENEFICIARIO / DESARROLLADOR,Producción de proteínas recombinantes a partir...,77065948-5,Pewman Innovation spa,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",28500000.0,7800000.0,5400000.0,NaN,77065948-5,True,PEWMAN INNOVATION SPA,77065948
4,4,5,2023.0,23CH-246125,Capital Humano,BENEFICIARIO / DESARROLLADOR,NGB-eDNA. Plataforma genética no-invasiva para...,77063937-9,Ecología y Genética Ambiental SpA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3500000.0,3500000.0,NaN,77063937-9,True,ECOLOGIA Y GENETICA AMBIENTAL SPA,77063937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10566,10566,1344,NaN,PI-998,Ley I+D,mandante,Eficacia y toxicidad del uso de una familia de...,77424780-7,EWOS CHILE ALIMENTOS LIMITADA,PERSONA JURIDICA COMERCIAL,D - INDUSTRIAS MANUFACTURERAS NO METALICAS,NaN,NaN,NaN,76835703.0,77424780-7,True,EWOS CHILE ALIMENTOS LIMITADA,77424780
10567,10567,1345,NaN,PI-99869,Ley I+D,mandante,Análisis Proteómico por Machine-Learning: Perf...,76221912-3,LABORATORIO PATHOVET LIMITADA,PERSONA JURIDICA COMERCIAL,O - SERVICIOS SOCIALES Y DE SALUD,NaN,NaN,NaN,23076375.0,76221912-3,True,LABORATORIO PATHOVET LIMITADA,76221912
10568,10568,1346,NaN,PI-99871,Ley I+D,mandante,Desarrollo de un proceso de recubrimiento o lí...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19770000.0,76082953-6,True,NANOTEC SA,76082953
10569,10569,1347,NaN,PI-99872,Ley I+D,mandante,Nuevos usos de nanopartículas metálicas: Desar...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19800000.0,76082953-6,True,NANOTEC SA,76082953


# Descargar datos desde PortalChile

In [ ]:
def get_data_from_rut(rut):
  headers = {
    "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
  }
  url1 = "https://www.portalchile.org/api/suggest/{rut}?opcion=R"

  # La primera llamada retorna bastantes datos sobre la institución, pero presenta
  # un problema: en muchos casos, la actividad económica está incorrecta, ya que
  # sólo muestra la primera de ellas. Esto es especialmente problemático para universidades,
  # que suelen tener muchas actividades.
  # Por ello, de la primera respuesta sólo nos interesa obtener la razón social,
  # con la cual podemos hacer una segunda llamada a otro endpoint, que trae información
  # mucho más detallada.

  response1 = requests.get(url1.format(rut=rut), headers=headers)
  if response1.status_code != 200:
    return None

  suggestions = response1.json()["sugerencia"]
  if len(suggestions) != 1:
    return None

  data1 = response1.json()["sugerencia"][0]["_source"]
  rut_without_dv = data1["rut"]
  name_normalized = "-".join(data1["razon_social"].lower().split(" "))

  url2 = "https://www.portalchile.org/_next/data/yZqGI3zv6BcEg5khJvRzg/empresa/{name}-{rut}.json?id={name}-{rut}"
  response2 = requests.get(url2.format(name=name_normalized, rut=rut_without_dv), headers=headers)
  if response2.status_code != 200:
    return None
  return response2.json()



In [ ]:
def download_wrapper(rut):
  save_path = os.path.join(SAVE_DIR, f"{rut}.json")
  if os.path.exists(save_path):
    with open(save_path) as fp:
      return json.load(fp)
  data_rut = get_data_from_rut(rut)
  if data_rut is None:
    return

  with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(data_rut, f, ensure_ascii=False, indent=4)
  return data_rut

In [ ]:
ruts_unique = df_corfo.loc[df_corfo["rut_valido"], "rut_sin_dv"].unique()
len(ruts_unique)

5600

In [ ]:
common_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Resultados/Común"
SAVE_DIR = os.path.join(data_dir, "Instituciones")
os.makedirs(save_dir, exist_ok=True)

In [ ]:
data_institutions = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
  futures = {
    executor.submit(download_wrapper, rut): rut
    for rut in ruts_unique
  }
  for future in tqdm(concurrent.futures.as_completed(futures)):
    rut = futures[future]
    data_institutions[rut] = future.result()

5600it [01:34, 59.51it/s] 


## Construir dataframe

In [ ]:
data = []
for rut, institution in data_institutions.items():
  if institution is None:
    continue
  data.append([
    rut,
    institution["pageProps"]["data"]["empresa_rut"],
    institution["pageProps"]["data"]["empresa_nombre"],
    institution["pageProps"]["data"]["empresa_tipo"],
    institution["pageProps"]["data"]["empresa_subtipo"],
    institution["pageProps"]["data"]["empresa_provincia"],
    institution["pageProps"]["data"]["empresa_comuna"],
    institution["pageProps"]["data"]["actividades"],
    len(institution["pageProps"]["data"]["actividades"])
  ])

df_info = pd.DataFrame(data, columns=["rut", "rut_dv", "nombre", "empresa_tipo", "empresa_subtipo", "provincia", "comuna", "actividades", "n_actividades"]).set_index("rut")
df_info

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades
rut,,,,,,,,
77155166,77155166-1,REBORN ELECTRIC MOTORS SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,CACHAPOAL,RANCAGUA,"[{'rut': '77155166', 'descripcion': 'Venta al ...",4
76415846,76415846-6,CONSULTORIAS Y SERVICIOS MINEROS INTEGRALES LI...,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,EL LOA,COPIAPO,"[{'rut': '76415846', 'descripcion': 'Investiga...",7
76055501,76055501-0,QIN TECHNOLOGY SPA.,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,LAS CONDES,"[{'rut': '76055501', 'descripcion': 'Actividad...",2
76940369,76940369-8,SEA GROUP SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,PROVIDENCIA,"[{'rut': '76940369', 'descripcion': 'Otras act...",6
76772215,76772215-K,QUEMPIN SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,VALPARAÍSO,VINA DEL MAR,"[{'rut': '76772215', 'descripcion': 'Fabricaci...",4
...,...,...,...,...,...,...,...,...
76092092,76092092-4,INMOBILIARIA HUEMUL LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SIN PROVINCIA,LAS CONDES,"[{'rut': '76092092', 'descripcion': 'Compra, v...",6
76035565,76035565-8,SOCIEDAD MINERA LAGUNAS BRAVAS LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '76035565', 'descripcion': 'Explotaci...",2
89696400,89696400-3,RESITER S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,MAIPO,PROVIDENCIA,"[{'rut': '89696400', 'descripcion': 'Transport...",11


## Consolidar datos de actividades

Intentaremos dividir las instituciones en las siguientes categorías:
1. Instituciones de educación superior
2. Empresas (públicas/privadas)
3. Estado
4. IPSFL

In [ ]:
df_info["final"] = None

In [ ]:
unique_activities = {}
for rut, list_activities in df_info["actividades"].items():
  for single_activity in list_activities:
    id_activity = single_activity["idactividad"]
    if id_activity in unique_activities:
      unique_activities[id_activity]["instituciones"].append(rut)
    else:
      unique_activities[id_activity] = {
        "descripcion": single_activity["descripcion"],
        "instituciones": [rut]
      }

df_activities = pd.DataFrame.from_dict(unique_activities, orient="index")
df_activities

,descripcion,instituciones
451001,Venta al por mayor de vehículos automotores,"[77155166, 76608025, 76836205, 96616070, 92475..."
259900,Fabricación de otros productos elaborados de m...,"[77155166, 77189740, 76425184, 94637000, 76239..."
452002,Mantenimiento y reparación de vehículos automo...,"[77155166, 96792430, 76809228, 96605870, 76531..."
711002,Empresas de servicios de ingeniería y activida...,"[77155166, 76940369, 76772215, 76300594, 77084..."
721000,Investigaciones y desarrollo experimental en e...,"[76415846, 77063937, 77131055, 79990550, 76234..."
...,...,...
143000,Fabricación de artículos de punto y ganchillo,[90914000]
491100,Transporte interurbano de pasajeros por ferroc...,[81148200]
071000,Extracción de minerales de hierro,[94638000]
107200,Elaboración de azúcar,[96772810]


### Instituciones de Educación Superior

#### Universidades

Para que una institución sea considerada universidad, debe:
1. Tener la actividad "Enseñanza superior en universidades públicas" o "Enseñanza superior en universidades públicas" (el substring "Enseñanza superior en universidades" sirve para ambos casos)
2. Tener la palabra _universidad_ en el nombre

In [ ]:
## df_activities.loc[...] return two rows, each one with a list of ruts.
## When we select .values, we end up with an array with two lists, so we merged them using itertools
ruts_universities = set(
    itertools.chain.from_iterable(
        df_activities.loc[df_activities["descripcion"].str.lower().str.contains("enseñanza superior en universidades"), "instituciones"].values
        )
    )
df_info.loc[(df_info.index.isin(ruts_universities)) &
            (df_info["nombre"].str.lower().str.contains("universidad")) &
            (df_info["final"].isnull()),
            "final"] = "Institución de Educación Superior"
df_info.loc[(df_info.index.isin(ruts_universities)) &
            (df_info["nombre"].str.lower().str.contains("universidad"))]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
60911000,60911000-7,UNIVERSIDAD DE SANTIAGO DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,EST CENTRAL,"[{'rut': '60911000', 'descripcion': 'Compra, v...",11,Institución de Educación Superior
60910000,60910000-1,UNIVERSIDAD DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60910000', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
71647500,71647500-K,UNIVERSIDAD BERNARDO OHIGGINS,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71647500', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81668700,81668700-4,UNIVERSIDAD TECNICA FEDERICO SANTA MARIA,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81668700', 'descripcion': 'Alquiler ...",9,Institución de Educación Superior
71633300,71633300-0,UNIVERSIDAD AUTONOMA DE CHILE,ORG. SIN FINES DE LUCRO,OTRA OSFL,CAUTÍN,TEMUCO,"[{'rut': '71633300', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
71540100,71540100-2,UNIVERSIDAD NACIONAL ANDRES BELLO,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71540100', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
60911006,60911006-6,UNIVERSIDAD DEL BIO BIO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,CONCEPCION,"[{'rut': '60911006', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior


#### Institutos Profesionales y Centros de Formación Técnica

In [ ]:
ruts_ips_cfts = set(itertools.chain.from_iterable(
    df_activities.loc[
        (df_activities["descripcion"].str.lower().str.contains("enseñanza superior en institutos profesionales")) |
        (df_activities["descripcion"].str.lower().str.contains("enseñanza superior en centros de formación")),
        "instituciones"].values)
    )
df_info.loc[(df_info.index.isin(ruts_ips_cfts)), ["nombre", "empresa_tipo", "empresa_subtipo", "n_actividades", "actividades"]]

,nombre,empresa_tipo,empresa_subtipo,n_actividades,actividades
rut,,,,,
77209932,ORIGO LAB SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,7,"[{'rut': '77209932', 'descripcion': 'Actividad..."
87845200,IACC S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,3,"[{'rut': '87845200', 'descripcion': 'Enseñanza..."
70777500,UNIVERSIDAD ARTURO PRAT,ORG. SIN FINES DE LUCRO,OTRA OSFL,7,"[{'rut': '70777500', 'descripcion': 'Centros m..."
72754700,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,1,"[{'rut': '72754700', 'descripcion': 'Enseñanza..."
71538400,INSTITUTO REGIONAL DE ADMINISTRACION DE EMPRESAS,ORG. SIN FINES DE LUCRO,OTRA OSFL,5,"[{'rut': '71538400', 'descripcion': 'Compra, v..."
77709380,INTERNATIONAL CENTER CAPACITACION SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,1,"[{'rut': '77709380', 'descripcion': 'Enseñanza..."
65071859,FUNDACION PARA EL TRABAJO UNIVERSIDAD ARTURO PRAT,ORG. SIN FINES DE LUCRO,FUNDACION,3,"[{'rut': '65071859', 'descripcion': 'Enseñanza..."
76181010,ASESORIAS PAREDES Y PAREDES LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,6,"[{'rut': '76181010', 'descripcion': 'Enseñanza..."
76088184,CAPACITACION INTEGRAL PARA EL DESARROLLO PROFE...,PERSONA JURIDICA COMERCIAL,EMPR. INDIVIDUAL RESP. LTDA.,2,"[{'rut': '76088184', 'descripcion': 'Enseñanza..."


Son IPs o CFTs:
- IACC S.A.
- FUND INSTITUTO PROFESIONAL DUOC UC
- INSTITUTO PROFESIONAL AIEP SPA
- INSTITUTO NACIONAL DE CAPACITACION PROFESIONAL
- CENTRO DE FORMACION TECNICA TEODORO WICKEL KLUWEN S A
- INSTITUTO PROFESIONAL LA ARAUCANA SPA


In [ ]:
names = ["IACC S.A.",
        "FUND INSTITUTO PROFESIONAL DUOC UC",
        "INSTITUTO PROFESIONAL AIEP SPA",
        "INSTITUTO NACIONAL DE CAPACITACION PROFESIONAL",
        "CENTRO DE FORMACION TECNICA TEODORO WICKEL KLUWEN S A",
        "INSTITUTO PROFESIONAL LA ARAUCANA SPA"]

df_info.loc[df_info["nombre"].isin(names), "final"] = "Institución de Educación Superior"
df_info.loc[df_info["nombre"].isin(names)]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
87845200,87845200-3,IACC S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,PROVIDENCIA,"[{'rut': '87845200', 'descripcion': 'Enseñanza...",3,Institución de Educación Superior
72754700,72754700-2,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '72754700', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
96621640,96621640-9,INSTITUTO PROFESIONAL AIEP SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,PROVIDENCIA,"[{'rut': '96621640', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
60711000,60711000-K,INSTITUTO NACIONAL DE CAPACITACION PROFESIONAL,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,VITACURA,"[{'rut': '60711000', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
96986340,96986340-5,CENTRO DE FORMACION TECNICA TEODORO WICKEL KLU...,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,CAUTÍN,TEMUCO,"[{'rut': '96986340', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
96635520,96635520-4,INSTITUTO PROFESIONAL LA ARAUCANA SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,SANTIAGO,"[{'rut': '96635520', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior


In [ ]:
df_info.loc[df_info["final"] == "Institución de Educación Superior"]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
60911000,60911000-7,UNIVERSIDAD DE SANTIAGO DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,EST CENTRAL,"[{'rut': '60911000', 'descripcion': 'Compra, v...",11,Institución de Educación Superior
60910000,60910000-1,UNIVERSIDAD DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60910000', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
71647500,71647500-K,UNIVERSIDAD BERNARDO OHIGGINS,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71647500', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81668700,81668700-4,UNIVERSIDAD TECNICA FEDERICO SANTA MARIA,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81668700', 'descripcion': 'Alquiler ...",9,Institución de Educación Superior
71633300,71633300-0,UNIVERSIDAD AUTONOMA DE CHILE,ORG. SIN FINES DE LUCRO,OTRA OSFL,CAUTÍN,TEMUCO,"[{'rut': '71633300', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
71540100,71540100-2,UNIVERSIDAD NACIONAL ANDRES BELLO,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71540100', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
60911006,60911006-6,UNIVERSIDAD DEL BIO BIO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,CONCEPCION,"[{'rut': '60911006', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior


### Empresas

Consideraremos empresas privadas y públicas


#### Empresas públicas

In [ ]:
df_soc = pd.read_excel(os.path.join(data_dir, "data_empresas_estatales.xlsx"))
df_soc["rut_sin_dv"] = df_soc["rut_sin_dv"].astype(str)
df_soc

,nombre,rut,rut_sin_dv
0,Empresa Portuaria Arica,61945700-5,61945700
1,Empresa Portuaria Iquique,61951300-2,61951300
2,Empresa Portuaria Antofagasta,73968300-9,73968300
3,Empresa Portuaria Coquimbo,61946300-5,61946300
4,Empresa Portuaria Valparaíso,61952700-3,61952700
5,Empresa Portuaria San Antonio,61960100-9,61960100
6,Empresa Portuaria Talcahuano San Vicente,61960300-1,61960300
7,Empresa Portuaria Puerto Montt,61950900-5,61950900
8,Empresa Portuaria Chacabuco,61959100-3,61959100
9,Empresa Portuaria Austral,61956700-5,61956700


Veamos ahora si hay empresas públicas en nuestro conjunto de datos

In [ ]:
df_info[df_info.index.isin(df_soc["rut_sin_dv"].values)]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
61960100,61960100-9,EMPRESA PORTUARIA SAN ANTONIO,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,SAN ANTONIO,SAN ANTONIO,"[{'rut': '61960100', 'descripcion': 'Otros ser...",4,None
61952700,61952700-3,EMPRESA PORTUARIA VALPARAISO,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,VALPARAÍSO,VALPARAISO,"[{'rut': '61952700', 'descripcion': 'Actividad...",4,None
70285500,70285500-4,ZONA FRANCA DE IQUIQUE S A,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,IQUIQUE,IQUIQUE,"[{'rut': '70285500', 'descripcion': 'Compra, v...",3,None
61960300,61960300-1,EMPRESA PORTUARIA TALCAHUANO SAN VICENTE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,TALCAHUANO,"[{'rut': '61960300', 'descripcion': 'Compra, v...",5,None
60806000,60806000-6,CASA DE MONEDA DE CHILE S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,SANTIAGO,EST CENTRAL,"[{'rut': '60806000', 'descripcion': 'Fabricaci...",5,None


Hay 5 empresas: Empresas Portuarias de San Antonio, Valparaíso y Talcahuano-San Vicente, Zona Franca de Iquique y Casa de Moneda

In [ ]:
df_info.loc[df_info.index.isin(df_soc["rut_sin_dv"].values), "final"] = "Empresa"
df_info.loc[df_info.index.isin(df_soc["rut_sin_dv"].values)]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
61960100,61960100-9,EMPRESA PORTUARIA SAN ANTONIO,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,SAN ANTONIO,SAN ANTONIO,"[{'rut': '61960100', 'descripcion': 'Otros ser...",4,Empresa
61952700,61952700-3,EMPRESA PORTUARIA VALPARAISO,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,VALPARAÍSO,VALPARAISO,"[{'rut': '61952700', 'descripcion': 'Actividad...",4,Empresa
70285500,70285500-4,ZONA FRANCA DE IQUIQUE S A,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,IQUIQUE,IQUIQUE,"[{'rut': '70285500', 'descripcion': 'Compra, v...",3,Empresa
61960300,61960300-1,EMPRESA PORTUARIA TALCAHUANO SAN VICENTE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,TALCAHUANO,"[{'rut': '61960300', 'descripcion': 'Compra, v...",5,Empresa
60806000,60806000-6,CASA DE MONEDA DE CHILE S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,SANTIAGO,EST CENTRAL,"[{'rut': '60806000', 'descripcion': 'Fabricaci...",5,Empresa


#### Empresas privadas

Empresas privadas serán todas aquellas que tengan tipo _PERSONA JURIDICA COMERCIAL_. De todas formas, primero revisaremos si existen instituciones con ese tipo y a las que ya le hayamos asignado clasificación.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "PERSONA JURIDICA COMERCIAL") & (~df_info["final"].isnull())]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
81518400,81518400-9,UNIVERSIDAD CATOLICA DEL NORTE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '81518400', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
87845200,87845200-3,IACC S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,PROVIDENCIA,"[{'rut': '87845200', 'descripcion': 'Enseñanza...",3,Institución de Educación Superior
61960100,61960100-9,EMPRESA PORTUARIA SAN ANTONIO,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,SAN ANTONIO,SAN ANTONIO,"[{'rut': '61960100', 'descripcion': 'Otros ser...",4,Empresa
61952700,61952700-3,EMPRESA PORTUARIA VALPARAISO,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,VALPARAÍSO,VALPARAISO,"[{'rut': '61952700', 'descripcion': 'Actividad...",4,Empresa
70285500,70285500-4,ZONA FRANCA DE IQUIQUE S A,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,IQUIQUE,IQUIQUE,"[{'rut': '70285500', 'descripcion': 'Compra, v...",3,Empresa
96621640,96621640-9,INSTITUTO PROFESIONAL AIEP SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,PROVIDENCIA,"[{'rut': '96621640', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
60806000,60806000-6,CASA DE MONEDA DE CHILE S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS ABIERTAS,SANTIAGO,EST CENTRAL,"[{'rut': '60806000', 'descripcion': 'Fabricaci...",5,Empresa
96986340,96986340-5,CENTRO DE FORMACION TECNICA TEODORO WICKEL KLU...,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,CAUTÍN,TEMUCO,"[{'rut': '96986340', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior


Existen instituciones con tipo _PERSONA JURIDICA COMERCIAL_ que clasificamos como Instituciones de Educación Superior. Debemos tener cuidado de no sobreescribir esa clasificación previa.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "PERSONA JURIDICA COMERCIAL") &
            (df_info["final"].isnull()),
            "final"] = "Empresa"
df_info.loc[(df_info["empresa_tipo"] == "PERSONA JURIDICA COMERCIAL")]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
77155166,77155166-1,REBORN ELECTRIC MOTORS SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,CACHAPOAL,RANCAGUA,"[{'rut': '77155166', 'descripcion': 'Venta al ...",4,Empresa
76415846,76415846-6,CONSULTORIAS Y SERVICIOS MINEROS INTEGRALES LI...,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,EL LOA,COPIAPO,"[{'rut': '76415846', 'descripcion': 'Investiga...",7,Empresa
76055501,76055501-0,QIN TECHNOLOGY SPA.,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,LAS CONDES,"[{'rut': '76055501', 'descripcion': 'Actividad...",2,Empresa
76940369,76940369-8,SEA GROUP SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,PROVIDENCIA,"[{'rut': '76940369', 'descripcion': 'Otras act...",6,Empresa
76772215,76772215-K,QUEMPIN SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,VALPARAÍSO,VINA DEL MAR,"[{'rut': '76772215', 'descripcion': 'Fabricaci...",4,Empresa
...,...,...,...,...,...,...,...,...,...
76000275,76000275-5,EXPORTADORA MAGNA TRADING S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,LAS CONDES,"[{'rut': '76000275', 'descripcion': 'Venta al ...",1,Empresa
76092092,76092092-4,INMOBILIARIA HUEMUL LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SIN PROVINCIA,LAS CONDES,"[{'rut': '76092092', 'descripcion': 'Compra, v...",6,Empresa
76035565,76035565-8,SOCIEDAD MINERA LAGUNAS BRAVAS LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '76035565', 'descripcion': 'Explotaci...",2,Empresa


Revisaremos no haber sobreescrito las IES con tipo _PERSONA JURIDICA COMERCIAL_.

In [ ]:
df_info.loc[["87845200", "81698900", "81518400"]]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
87845200,87845200-3,IACC S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,PROVIDENCIA,"[{'rut': '87845200', 'descripcion': 'Enseñanza...",3,Institución de Educación Superior
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
81518400,81518400-9,UNIVERSIDAD CATOLICA DEL NORTE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '81518400', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior


### Estado

Asignaremos a Estado todas las instituciones que cuenten con las siguientes características:
1. Tener tipo _INSTITUCIONES FISCALES_ O _MUNICIPALIDADES_
2. No tener clasificación previa asignada.

Primero, revisaremos qué instituciones tienen tipo _INSITUCIONES FISCALES_ o _MUNICIPALIDAD_ y SÍ tienen clasificación previa.

In [ ]:
df_info[(df_info["empresa_tipo"].isin(["INSTITUCIONES FISCALES", "MUNICIPALIDADES"])) &
        (~df_info["final"].isnull())]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
60911000,60911000-7,UNIVERSIDAD DE SANTIAGO DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,EST CENTRAL,"[{'rut': '60911000', 'descripcion': 'Compra, v...",11,Institución de Educación Superior
60910000,60910000-1,UNIVERSIDAD DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60910000', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior
60911006,60911006-6,UNIVERSIDAD DEL BIO BIO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,CONCEPCION,"[{'rut': '60911006', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
61980530,61980530-5,UNIVERSIDAD DE OHIGGINS,INSTITUCIONES FISCALES,ORG. EDUCACION SUPERIOR,CACHAPOAL,RANCAGUA,"[{'rut': '61980530', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
60921000,60921000-1,UNIVERSIDAD DE VALPARAISO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,VALPARAÍSO,VALPARAISO,"[{'rut': '60921000', 'descripcion': 'Compra, v...",12,Institución de Educación Superior
87912900,87912900-1,UNIVERSIDAD DE LA FRONTERA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CAUTÍN,TEMUCO,"[{'rut': '87912900', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
61980520,61980520-8,UNIVERSIDAD DE AYSEN,INSTITUCIONES FISCALES,ORG. EDUCACION SUPERIOR,COIHAIQUE,COYHAIQUE,"[{'rut': '61980520', 'descripcion': 'Compra, v...",2,Institución de Educación Superior
60711000,60711000-K,INSTITUTO NACIONAL DE CAPACITACION PROFESIONAL,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,VITACURA,"[{'rut': '60711000', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
61960300,61960300-1,EMPRESA PORTUARIA TALCAHUANO SAN VICENTE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,TALCAHUANO,"[{'rut': '61960300', 'descripcion': 'Compra, v...",5,Empresa


Nuevamente, debemos tener cuidado de no sobreeescribir esas instituciones.

In [ ]:
df_info.loc[(df_info["empresa_tipo"].isin(["INSTITUCIONES FISCALES", "MUNICIPALIDADES"])) &
            (df_info["final"].isnull()),
            "final"] = "Estado"
df_info.loc[(df_info["empresa_tipo"].isin(["INSTITUCIONES FISCALES", "MUNICIPALIDADES"]))]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
60911000,60911000-7,UNIVERSIDAD DE SANTIAGO DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,EST CENTRAL,"[{'rut': '60911000', 'descripcion': 'Compra, v...",11,Institución de Educación Superior
60910000,60910000-1,UNIVERSIDAD DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60910000', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior
61704000,61704000-K,CORP NACIONAL DEL COBRE DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '61704000', 'descripcion': 'Venta al ...",9,Estado
60911006,60911006-6,UNIVERSIDAD DEL BIO BIO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,CONCEPCION,"[{'rut': '60911006', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
61980530,61980530-5,UNIVERSIDAD DE OHIGGINS,INSTITUCIONES FISCALES,ORG. EDUCACION SUPERIOR,CACHAPOAL,RANCAGUA,"[{'rut': '61980530', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
...,...,...,...,...,...,...,...,...,...
69251900,69251900-0,ILUSTRE MUNICIPALIDAD DE ALTO DEL CARMEN,MUNICIPALIDADES,MUNICIPALIDAD,HUASCO,ALTO DEL CARMEN,"[{'rut': '69251900', 'descripcion': 'Actividad...",1,Estado
61211000,61211000-K,INSTITUTO NACIONAL DE HIDRAULICA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '61211000', 'descripcion': 'Investiga...",2,Estado
69170400,69170400-9,I MUNICIPALIDAD DE QUILLECO,MUNICIPALIDADES,MUNICIPALIDAD,BIOBÍO,QUILLECO,"[{'rut': '69170400', 'descripcion': 'Otras act...",2,Estado


Vemos que en la etiqueta _final_ aparece tanto Estado como Institución de Educación Superior, indicando que los datos previos no se sobreescribieron.

### Instituciones Privadas Sin Fines de Lucro (IPSFL)

Veamos primero las organizaciones que aún no tienen clasificación

In [ ]:
df_info.loc[df_info["final"].isnull()].groupby(["empresa_tipo", "empresa_subtipo"])["nombre"].count()

empresa_tipo             empresa_subtipo                     
ORG. SIN FINES DE LUCRO  ASOC. GREMIAL                            28
                         CLUB DEPORTIVO                            1
                         COOPERATIVA                              26
                         CORPORACION                              49
                         FUNDACION                                68
                         JUNTA DE VECINOS, ORG. COMUNITARIA        8
                         OTRA OSFL                               169
                         SINDICATO                                 1
SIN PER. JURIDICA        OTRAS ORGANIZACIONES SIN P. JURIDICA      1
                         SOCIEDADES DE HECHO                       3
                         SUCESIONES O COMUNIDADES HERED            1
SOCIEDADES EXTRANJERAS   AGENCIA                                   1
                         ENTIDAD SIN RESIDENCIA                    4
                         OTRO ESTABLECIMI

Revisemos las instituciones con tipo _ORG. SIN FINES DE LUCRO_

In [ ]:
df_info.loc[df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO"]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
65081283,65081283-2,FUNDACION LEITAT CHILE,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,PROVIDENCIA,"[{'rut': '65081283', 'descripcion': 'Actividad...",1,None
65088122,65088122-2,FUNDACION PARQUE CIENTIFICO TECNOLOGICO DE LA ...,ORG. SIN FINES DE LUCRO,FUNDACION,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '65088122', 'descripcion': 'Alquiler ...",4,None
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
71647500,71647500-K,UNIVERSIDAD BERNARDO OHIGGINS,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71647500', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81668700,81668700-4,UNIVERSIDAD TECNICA FEDERICO SANTA MARIA,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81668700', 'descripcion': 'Alquiler ...",9,Institución de Educación Superior
...,...,...,...,...,...,...,...,...,...
70265300,70265300-2,ASOC GREMIAL DE LA UNION INDUST DE PANAD,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '70265300', 'descripcion': 'Edición d...",2,None
65052193,65052193-5,FUNDACION IMPULSORA DE UN NUEVO SECTOR EN LA E...,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,LAS CONDES,"[{'rut': '65052193', 'descripcion': 'Venta al ...",2,None
65168990,65168990-2,FUNDACION ACCION RESPONSABILIDAD SOCIAL EMPRES...,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '65168990', 'descripcion': 'Fundacion...",5,None


Como siempre, debemos tener cuidado con las instituciones con clasificación previa.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO") &
            (~df_info["final"].isnull())]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
71647500,71647500-K,UNIVERSIDAD BERNARDO OHIGGINS,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71647500', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81668700,81668700-4,UNIVERSIDAD TECNICA FEDERICO SANTA MARIA,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81668700', 'descripcion': 'Alquiler ...",9,Institución de Educación Superior
71633300,71633300-0,UNIVERSIDAD AUTONOMA DE CHILE,ORG. SIN FINES DE LUCRO,OTRA OSFL,CAUTÍN,TEMUCO,"[{'rut': '71633300', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
71540100,71540100-2,UNIVERSIDAD NACIONAL ANDRES BELLO,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71540100', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
71918300,71918300-K,UNIVERSIDAD CATOLICA DEL MAULE,ORG. SIN FINES DE LUCRO,OTRA OSFL,TALCA,TALCA,"[{'rut': '71918300', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior
71918700,71918700-5,UNIVERSIDAD CATOLICA DE TEMUCO,ORG. SIN FINES DE LUCRO,OTRA OSFL,CAUTÍN,TEMUCO,"[{'rut': '71918700', 'descripcion': 'Enseñanza...",14,Institución de Educación Superior
81669200,81669200-8,PONTIFICIA UNIVERSIDAD CATOLICA DE VALPARAISO,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81669200', 'descripcion': 'Suministr...",7,Institución de Educación Superior
70783100,70783100-6,UNIVERSIDAD DE LA SERENA,ORG. SIN FINES DE LUCRO,OTRA OSFL,ELQUI,LA SERENA,"[{'rut': '70783100', 'descripcion': 'Suministr...",6,Institución de Educación Superior


Aparecen IES y una empresa estatal; no debemos sobreescribirlas.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO") &
            (df_info["final"].isnull()),
            "final"] = "Institución Privada Sin Fines de Lucro"
df_info.loc[(df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO")]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
65081283,65081283-2,FUNDACION LEITAT CHILE,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,PROVIDENCIA,"[{'rut': '65081283', 'descripcion': 'Actividad...",1,Institución Privada Sin Fines de Lucro
65088122,65088122-2,FUNDACION PARQUE CIENTIFICO TECNOLOGICO DE LA ...,ORG. SIN FINES DE LUCRO,FUNDACION,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '65088122', 'descripcion': 'Alquiler ...",4,Institución Privada Sin Fines de Lucro
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
71647500,71647500-K,UNIVERSIDAD BERNARDO OHIGGINS,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71647500', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81668700,81668700-4,UNIVERSIDAD TECNICA FEDERICO SANTA MARIA,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81668700', 'descripcion': 'Alquiler ...",9,Institución de Educación Superior
...,...,...,...,...,...,...,...,...,...
70265300,70265300-2,ASOC GREMIAL DE LA UNION INDUST DE PANAD,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '70265300', 'descripcion': 'Edición d...",2,Institución Privada Sin Fines de Lucro
65052193,65052193-5,FUNDACION IMPULSORA DE UN NUEVO SECTOR EN LA E...,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,LAS CONDES,"[{'rut': '65052193', 'descripcion': 'Venta al ...",2,Institución Privada Sin Fines de Lucro
65168990,65168990-2,FUNDACION ACCION RESPONSABILIDAD SOCIAL EMPRES...,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '65168990', 'descripcion': 'Fundacion...",5,Institución Privada Sin Fines de Lucro


Aparecen tanto IPSFL como IES, lo cual muestra que no hubo sobreescritura.

### Otras instituciones

Veamos las instituciones que hasta ahora no han sido categorizadas

In [ ]:
df_info[df_info["final"].isnull()]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
53290880,53290880-9,SUC LUIS ALFONSO DURAN ORELLANA,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,COLCHAGUA,PLACILLA,"[{'rut': '53290880', 'descripcion': 'Compra, v...",1,None
53323226,53323226-4,FUNDACION BASURA,SIN PER. JURIDICA,OTRAS ORGANIZACIONES SIN P. JURIDICA,SANTIAGO,PROVIDENCIA,"[{'rut': '53323226', 'descripcion': 'Otras act...",3,None
59087530,59087530-9,ECOMETALES LIMITED AGENCIA EN CHILE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,PROVIDENCIA,"[{'rut': '59087530', 'descripcion': 'Recuperac...",2,None
59198140,59198140-4,CENTRO DESARROLLO Y PESCA SUSTENTABLE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,VALDIVIA,VALDIVIA,"[{'rut': '59198140', 'descripcion': 'Actividad...",2,None
53306574,53306574-0,SOCIEDAD AGRICOLA Y VIVERO SAN RAFAEL LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,LOS ANDES,LOS ANDES,"[{'rut': '53306574', 'descripcion': 'Cultivo d...",1,None
50741230,50741230-0,ASOCIACION CHILE DISENO ASOCIACION GREMIAL,SIN PER. JURIDICA,SOCIEDADES DE HECHO,SANTIAGO,SANTIAGO,"[{'rut': '50741230', 'descripcion': 'Otras act...",4,None
50333990,50333990-0,IMPACTO PUBLICITARIO LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,CAUTÍN,TEMUCO,"[{'rut': '50333990', 'descripcion': 'Servicios...",2,None
81148200,81148200-5,FERROCARRIL DE ANTOFAGASTA A BOLIVIA,SOCIEDADES EXTRANJERAS,OTRO ESTABLECIMIENTO PERMANENTE,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '81148200', 'descripcion': 'Otras act...",4,None
59069860,59069860-1,ACCIONA CONSTRUCCION S.A. AGENCIA CHILE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,LAS CONDES,"[{'rut': '59069860', 'descripcion': 'Servicios...",6,None


Las sociedades extranjeras y los organismos internacionales los dejaremos como "Internacional".

In [ ]:
df_info.loc[(df_info["empresa_tipo"].isin(["SOCIEDADES EXTRANJERAS", "ORGANISMOS INTERNACIONALES"])) &
            (df_info["final"].isnull()),
            "final"] = "Internacional"
df_info.loc[df_info["final"] == "Internacional"]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
59087530,59087530-9,ECOMETALES LIMITED AGENCIA EN CHILE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,PROVIDENCIA,"[{'rut': '59087530', 'descripcion': 'Recuperac...",2,Internacional
59198140,59198140-4,CENTRO DESARROLLO Y PESCA SUSTENTABLE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,VALDIVIA,VALDIVIA,"[{'rut': '59198140', 'descripcion': 'Actividad...",2,Internacional
81148200,81148200-5,FERROCARRIL DE ANTOFAGASTA A BOLIVIA,SOCIEDADES EXTRANJERAS,OTRO ESTABLECIMIENTO PERMANENTE,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '81148200', 'descripcion': 'Otras act...",4,Internacional
59069860,59069860-1,ACCIONA CONSTRUCCION S.A. AGENCIA CHILE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,LAS CONDES,"[{'rut': '59069860', 'descripcion': 'Servicios...",6,Internacional
59281960,59281960-0,LABORELEC LATIN AMERICA,SOCIEDADES EXTRANJERAS,AGENCIA,SANTIAGO,PROVIDENCIA,"[{'rut': '59281960', 'descripcion': 'Venta al ...",2,Internacional
59108780,59108780-0,ENYSE AGENCIA CHILE S A,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,SANTIAGO,"[{'rut': '59108780', 'descripcion': 'Construcc...",1,Internacional


Veamos ahora los casos restantes

In [ ]:
df_info.loc[df_info["final"].isnull()]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
53290880,53290880-9,SUC LUIS ALFONSO DURAN ORELLANA,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,COLCHAGUA,PLACILLA,"[{'rut': '53290880', 'descripcion': 'Compra, v...",1,None
53323226,53323226-4,FUNDACION BASURA,SIN PER. JURIDICA,OTRAS ORGANIZACIONES SIN P. JURIDICA,SANTIAGO,PROVIDENCIA,"[{'rut': '53323226', 'descripcion': 'Otras act...",3,None
53306574,53306574-0,SOCIEDAD AGRICOLA Y VIVERO SAN RAFAEL LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,LOS ANDES,LOS ANDES,"[{'rut': '53306574', 'descripcion': 'Cultivo d...",1,None
50741230,50741230-0,ASOCIACION CHILE DISENO ASOCIACION GREMIAL,SIN PER. JURIDICA,SOCIEDADES DE HECHO,SANTIAGO,SANTIAGO,"[{'rut': '50741230', 'descripcion': 'Otras act...",4,None
50333990,50333990-0,IMPACTO PUBLICITARIO LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,CAUTÍN,TEMUCO,"[{'rut': '50333990', 'descripcion': 'Servicios...",2,None


"ASOCIACION CHILE DISENO ASOCIACION GREMIAL" y "FUNDACION BASURA" son IPSFL. "SOCIEDAD AGRICOLA Y VIVERO SAN RAFAEL LIMITADA" e "IMPACTO PUBLICITARIO LIMITADA" son empresas.

In [ ]:
df_info.loc[["53323226", "50741230"], "final"] = "Institución Privada Sin Fines de Lucro"
df_info.loc[["50333990", "53306574"], "final"] = "Empresa"
df_info.loc[["53290880"], "final"] = "Otra"
df_info.loc[["53323226", "50741230", "50333990", "53306574", "53290880"]]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
53323226,53323226-4,FUNDACION BASURA,SIN PER. JURIDICA,OTRAS ORGANIZACIONES SIN P. JURIDICA,SANTIAGO,PROVIDENCIA,"[{'rut': '53323226', 'descripcion': 'Otras act...",3,Institución Privada Sin Fines de Lucro
50741230,50741230-0,ASOCIACION CHILE DISENO ASOCIACION GREMIAL,SIN PER. JURIDICA,SOCIEDADES DE HECHO,SANTIAGO,SANTIAGO,"[{'rut': '50741230', 'descripcion': 'Otras act...",4,Institución Privada Sin Fines de Lucro
50333990,50333990-0,IMPACTO PUBLICITARIO LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,CAUTÍN,TEMUCO,"[{'rut': '50333990', 'descripcion': 'Servicios...",2,Empresa
53306574,53306574-0,SOCIEDAD AGRICOLA Y VIVERO SAN RAFAEL LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,LOS ANDES,LOS ANDES,"[{'rut': '53306574', 'descripcion': 'Cultivo d...",1,Empresa
53290880,53290880-9,SUC LUIS ALFONSO DURAN ORELLANA,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,COLCHAGUA,PLACILLA,"[{'rut': '53290880', 'descripcion': 'Compra, v...",1,Otra


In [ ]:
df_info["final"].value_counts()

Empresa                                   4414
Institución Privada Sin Fines de Lucro     352
Estado                                      99
Institución de Educación Superior           45
Internacional                                6
Otra                                         1
Name: final, dtype: int64

In [ ]:
df_final = df_info.rename(columns={"final": "tipo_institucion_final"})[["rut_dv", "nombre", "empresa_tipo", "empresa_subtipo", "provincia", "comuna", "tipo_institucion_final"]].reset_index()
df_final

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,77155166,77155166-1,REBORN ELECTRIC MOTORS SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,CACHAPOAL,RANCAGUA,Empresa
1,76415846,76415846-6,CONSULTORIAS Y SERVICIOS MINEROS INTEGRALES LI...,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,EL LOA,COPIAPO,Empresa
2,76055501,76055501-0,QIN TECHNOLOGY SPA.,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,LAS CONDES,Empresa
3,76940369,76940369-8,SEA GROUP SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,PROVIDENCIA,Empresa
4,76772215,76772215-K,QUEMPIN SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,VALPARAÍSO,VINA DEL MAR,Empresa
...,...,...,...,...,...,...,...,...
4912,76092092,76092092-4,INMOBILIARIA HUEMUL LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SIN PROVINCIA,LAS CONDES,Empresa
4913,76035565,76035565-8,SOCIEDAD MINERA LAGUNAS BRAVAS LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,Empresa
4914,89696400,89696400-3,RESITER S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,MAIPO,PROVIDENCIA,Empresa
4915,59108780,59108780-0,ENYSE AGENCIA CHILE S A,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,SANTIAGO,Internacional


# Descargar datos desde Registros 19862

In [ ]:
missing_ruts = set(df_corfo["rut_sin_dv"].unique()) - set(df_final["rut"])
missing_ruts

{'-',
 '09194729',
 '10008072',
 '10011586',
 '10061415',
 '10064598',
 '10065507',
 '10080263',
 '10091693',
 '10109169',
 '10123635',
 '10173704',
 '10179187',
 '10216303',
 '10275991',
 '10308090',
 '10331889',
 '10337570',
 '10350777',
 '10358509',
 '10364295',
 '10370792',
 '10417568',
 '10473158',
 '10512893',
 '10585123',
 '10587588',
 '10620351',
 '10621346',
 '10633391',
 '10658101',
 '10663863',
 '10666189',
 '10701799',
 '10717804',
 '10733857',
 '10760809',
 '10780830',
 '10782071',
 '10805826',
 '10808405',
 '10839580',
 '10844297',
 '10908391',
 '10910364',
 '10946055',
 '10979001',
 '10981470',
 '10983781',
 '11111111',
 '11342652',
 '11344129',
 '11344584',
 '11345695',
 '11428220',
 '11485227',
 '11592546',
 '11612251',
 '11620946',
 '11672368',
 '11682564',
 '11707951',
 '11736643',
 '11841936',
 '11843804',
 '11852294',
 '11912700',
 '11935748',
 '11978256',
 '12039904',
 '12106734',
 '12133655',
 '12306340',
 '12312643',
 '12373240',
 '12401090',
 '12403640',
 '1243

In [ ]:
def get_data_from_registros1982(rut):
  url = f"https://registros19862.gob.cl/institucion/{rut}/mas"
  response = requests.get(url)
  if response.status_code != 200:
    return None
  result = {}
  soup = BeautifulSoup(response.text)
  payload = soup.find("section", class_="payload")
  name, type_header = [txt.strip() for txt in payload.find("div", class_="title").h1.text.split("\n") if txt.strip() != ""]
  result["Nombre"] = name
  result["Tipo 1"] = type_header
  rows = payload.find("table").find_all("tr")
  for row in rows:
    attr_name = row.th.text.strip()
    value = row.td.text.strip()
    result[attr_name] = value
  return result

In [ ]:
results = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
  futures = {
    executor.submit(get_data_from_registros1982, rut): rut
    for rut in list(missing_ruts)
  }
  for future in tqdm(concurrent.futures.as_completed(futures)):
    rut = futures[future]
    results[rut] = future.result()

686it [01:38,  6.93it/s]


## Construir dataframe

In [ ]:
results_valid = {rut: data for rut, data in results.items() if data is not None}

In [ ]:
len(results_valid)

161

In [ ]:
df_data = pd.DataFrame.from_dict(results_valid, orient="index")
df_data

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio
76794098,SERVICIOS TECNOLÓGICOS ALEJANDRO SANTIAGO PÉRE...,INSTITUCIÓN PRIVADA,76.794.098-K,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-01-2020 20:05,NaN
17349191,BERNARDITA OLIVOS URRUTIA,INSTITUCIÓN PRIVADA,17.349.191-3,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,10-06-2021 16:10,NaN
72187900,JUNTA DE VECINOS Nº11 VICTOR DOMINGO SILVA,INSTITUCIÓN PRIVADA,72.187.900-3,Agrupaciones,Otros,DECRETO 5731 DE FECHA 20 DE OCTUBRE DE 1994 ot...,$ 0,$ 0,$ 0,LOS CIPRECES 1089\n ...,23-06-2005 19:01,Representante legal\nJAQUELINE PATRICIA RAIN ...
76967211,RACIMOS SPA,INSTITUCIÓN PRIVADA,76.967.211-7,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,25-04-2022 18:13,NaN
77520996,BROLLY SPA,INSTITUCIÓN PRIVADA,77.520.996-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,13-09-2022 13:35,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77692848,MAHARO RAPA,INSTITUCIÓN PRIVADA,77.692.848-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-09-2023 11:19,NaN
8352508,JUSTO LORENZO GARCIA CAMPOS,INSTITUCIÓN PRIVADA,8.352.508-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,23-09-2020 15:03,NaN
13767547,PABLO LOPEZ URBINA,INSTITUCIÓN PRIVADA,13.767.547-1,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,24-11-2020 19:08,NaN
4335278,IRIS AVENDAÑO BRICEÑO,INSTITUCIÓN PRIVADA,4.335.278-4,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,IDAHUILLO S/N\n ...,28-06-2007 02:03,NaN


## Consolidar datos de actividades

Nuevamente, los dividiremos en
1. Instituciones de Educación Superior
2. Empresas
3. Estado
4. IPSFL
5. Otras

#### Instituciones de Educación Superior

Dado que son pocos datos, podemos revisarlos manualmente. Así, nos fijamos que no hay ninguno que corresponda a una IES, por lo cual no hay nada que hacer aquí

#### Empresas

In [ ]:
df_data["Tipo institución"].unique()

array(['Empresas', 'Agrupaciones', 'Juntas de Vecinos',
       'Fundaciones (Ej: CEMA Chile, Integra, Cruz Roja, Scout, etc.)',
       'Asociaciones', 'ONG',
       'Corporaciones (Ej: Educación, Salud, Cultura, etc.)',
       'Clubes (Ej: Deportivo, Juvenil, Adulto Mayor, etc.)',
       'Centros (Ej: Madres, Padres, Apoderados, Alumnos, Estudio, Investigación, etc.)',
       'Uniones comunales de juntas de vecinos'], dtype=object)

Caracterizaremos como empresas a todas aquellas instituciones que tengan el valor `Empresas` en la columna `Tipo institución`

In [ ]:
df_data["tipo_institucion_final"] = None
df_data.loc[df_data["Tipo institución"] == "Empresas", "tipo_institucion_final"] = "Empresa"
df_data

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio,tipo_institucion_final
76794098,SERVICIOS TECNOLÓGICOS ALEJANDRO SANTIAGO PÉRE...,INSTITUCIÓN PRIVADA,76.794.098-K,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-01-2020 20:05,NaN,Empresa
17349191,BERNARDITA OLIVOS URRUTIA,INSTITUCIÓN PRIVADA,17.349.191-3,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,10-06-2021 16:10,NaN,Empresa
72187900,JUNTA DE VECINOS Nº11 VICTOR DOMINGO SILVA,INSTITUCIÓN PRIVADA,72.187.900-3,Agrupaciones,Otros,DECRETO 5731 DE FECHA 20 DE OCTUBRE DE 1994 ot...,$ 0,$ 0,$ 0,LOS CIPRECES 1089\n ...,23-06-2005 19:01,Representante legal\nJAQUELINE PATRICIA RAIN ...,None
76967211,RACIMOS SPA,INSTITUCIÓN PRIVADA,76.967.211-7,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,25-04-2022 18:13,NaN,Empresa
77520996,BROLLY SPA,INSTITUCIÓN PRIVADA,77.520.996-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,13-09-2022 13:35,NaN,Empresa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77692848,MAHARO RAPA,INSTITUCIÓN PRIVADA,77.692.848-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-09-2023 11:19,NaN,Empresa
8352508,JUSTO LORENZO GARCIA CAMPOS,INSTITUCIÓN PRIVADA,8.352.508-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,23-09-2020 15:03,NaN,Empresa
13767547,PABLO LOPEZ URBINA,INSTITUCIÓN PRIVADA,13.767.547-1,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,24-11-2020 19:08,NaN,Empresa
4335278,IRIS AVENDAÑO BRICEÑO,INSTITUCIÓN PRIVADA,4.335.278-4,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,IDAHUILLO S/N\n ...,28-06-2007 02:03,NaN,Empresa


#### Estado



In [ ]:
df_data[df_data["Tipo 1"] == "SERVICIO PÚBLICO"]

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio,tipo_institucion_final


No hay nada en Servicio Público

#### IPSFL

Serán todas aquellas que tengan el tipo:
- Agrupaciones
- ONG
- Asociaciones
- Fundaciones (Ej: CEMA Chile, Integra, Cruz Roja, Scout, etc.)
- Comités
- Corporaciones (Ej: Educación, Salud, Cultura, etc.)
- Uniones comunales de organizaciones funcionales
- Centros (Ej: Madres, Padres, Apoderados, Alumnos, Estudio, Investigación, etc.)
- Juntas de Vecinos
- Uniones comunales de juntas de vecinos
- Clubes (Ej: Deportivo, Juvenil, Adulto Mayor, etc.)


In [ ]:
ipsfl_types = ['Agrupaciones', 'ONG', 'Asociaciones',
               'Fundaciones (Ej: CEMA Chile, Integra, Cruz Roja, Scout, etc.)',
               'Comités', 'Corporaciones (Ej: Educación, Salud, Cultura, etc.)',
               'Uniones comunales de organizaciones funcionales',
               'Centros (Ej: Madres, Padres, Apoderados, Alumnos, Estudio, Investigación, etc.)',
               'Juntas de Vecinos',
               'Clubes (Ej: Deportivo, Juvenil, Adulto Mayor, etc.)',
               'Uniones comunales de juntas de vecinos']
df_data.loc[df_data["Tipo institución"].isin(ipsfl_types), "tipo_institucion_final"] = "Institución Privada Sin Fines de Lucro"
df_data.loc[df_data["Tipo institución"].isin(ipsfl_types)]

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio,tipo_institucion_final
72187900,JUNTA DE VECINOS Nº11 VICTOR DOMINGO SILVA,INSTITUCIÓN PRIVADA,72.187.900-3,Agrupaciones,Otros,DECRETO 5731 DE FECHA 20 DE OCTUBRE DE 1994 ot...,$ 0,$ 0,$ 0,LOS CIPRECES 1089\n ...,23-06-2005 19:01,Representante legal\nJAQUELINE PATRICIA RAIN ...,Institución Privada Sin Fines de Lucro
72913800,JUNTA DE VECINOS N°43 DE CATRUMAN,INSTITUCIÓN PRIVADA,72.913.800-2,Juntas de Vecinos,Gestión organizacional,46 otorgada por LA ILUSTRE MUNICIPALIDAD DE ANCUD,$ 0,$ 0,$ 0,SECTOR RURAL DE CATRUMAN S/N\n ...,11-09-2007 10:05,Presidente(a)\nPIA ALEJANDRA BARRIA BARRIA\n15...,Institución Privada Sin Fines de Lucro
10983781,RENATE CECILIA DOCKENDORFF AGUILERA,INSTITUCIÓN PRIVADA,10.983.781-4,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,SANTIAGO,17-04-2009 11:39,NaN,Institución Privada Sin Fines de Lucro
65049692,FUNDACIÓN DOWN 21 CHLE GRUPO DOWN 21 CHILE,INSTITUCIÓN PRIVADA,65.049.692-2,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Discapacitados,4834 otorgada por ministerio de justicia,$ 5 000 000,$ 100 000,$ 6 000 000,INES RECREO 345\n ...,10-12-2012 21:59,Secretario(a)\nLILIANA RAMOS ABADIE\n11.491.66...,Institución Privada Sin Fines de Lucro
8073000,LUZMIRA MONICA RODRIGUEZ FICA,INSTITUCIÓN PRIVADA,8.073.000-4,Agrupaciones,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,SANTIAGO,28-06-2007 02:26,NaN,Institución Privada Sin Fines de Lucro
65374320,SINDICATO DE TRABAJADORAS INDEPENDIENTES RECOL...,INSTITUCIÓN PRIVADA,65.374.320-3,Agrupaciones,Apoyo y fomento productivo,1002380 otorgada por inspeccion del trabajo,$ 8 500 000,$ 100 000,$ 8 600 000,CALLE 1 SIN NUMERO\nN\n ...,18-10-2007 13:56,Representante legal\nPATRICIA SOLEDAD MUNDACA ...,Institución Privada Sin Fines de Lucro
75895100,ASOCIACION DE PRODUCTORES EXPORTADORES DE BULB...,INSTITUCIÓN PRIVADA,75.895.100-6,Asociaciones,Otros,3111 otorgada por UNIDAD DE ASOCIACIONES GRMIA...,$ 1 003 657,$ 1 987 613,$ -983 956,RUTA T 205 KM 34\n ...,03-10-2017 15:26,Representante legal\nJORGE BUSCHMANN REHBEIN\...,Institución Privada Sin Fines de Lucro
65144787,ASOCIACION GREMIAL DE PRODUCTORES DE SEMILLA D...,INSTITUCIÓN PRIVADA,65.144.787-9,Asociaciones,Apoyo y fomento productivo,328-10 otorgada por 23 Octubre del 2014,$ 0,$ 0,$ 0,CAMILO HERRIQUEZ\n ...,17-11-2021 11:26,Presidente(a)\nHERIBERTO PATRICIO GUENEL BARR...,Institución Privada Sin Fines de Lucro
10587588,EUGENIO RODRIGO BENAVENTE HORMAZABAL,INSTITUCIÓN PRIVADA,10.587.588-6,Agrupaciones,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,TEMUCO,28-06-2007 02:03,NaN,Institución Privada Sin Fines de Lucro
4954010,MANUEL JOSE CALVO LARRAIN,INSTITUCIÓN PRIVADA,4.954.010-8,Agrupaciones,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,RANCAGUA,06-02-2009 13:14,NaN,Institución Privada Sin Fines de Lucro


#### Internacional

Serán aquellos que tengan como tipo `Organismo Internacional`

In [ ]:
df_data.loc[df_data["Tipo institución"] == 'Organismo Internacional']

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio,tipo_institucion_final


No hay organizaciones internacionales

#### Otros

Veamos si hay organizaciones sin tipo_institucion_final.

In [ ]:
df_data[df_data["tipo_institucion_final"].isnull()]

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio,tipo_institucion_final


Están todas listas :)

### Más procesamiento

Debemos eliminar caracteres extraños dentro de los nombres de las instituciones.

In [ ]:
df_data["Nombre normalizado"] = df_data["Nombre"].apply(unidecode)

Parece funcionar bien. Revisemos los casos donde `Nombre` es distinto a `Nombre normalizado`

In [ ]:
df_data.loc[df_data["Nombre"] != df_data["Nombre normalizado"], ["Nombre", "Nombre normalizado"]]

,Nombre,Nombre normalizado
76794098,SERVICIOS TECNOLÓGICOS ALEJANDRO SANTIAGO PÉRE...,SERVICIOS TECNOLOGICOS ALEJANDRO SANTIAGO PERE...
72187900,JUNTA DE VECINOS Nº11 VICTOR DOMINGO SILVA,JUNTA DE VECINOS No11 VICTOR DOMINGO SILVA
72913800,JUNTA DE VECINOS N°43 DE CATRUMAN,JUNTA DE VECINOS Ndeg43 DE CATRUMAN
77395435,INSUMOS AGRÍCOLAS Y SERVICIOS DE INV. SPA,INSUMOS AGRICOLAS Y SERVICIOS DE INV. SPA
65049692,FUNDACIÓN DOWN 21 CHLE GRUPO DOWN 21 CHILE,FUNDACION DOWN 21 CHLE GRUPO DOWN 21 CHILE
76769768,DISEÑO Y ARQUITECTURA SUSTENTABLE DAS8 SPA,DISENO Y ARQUITECTURA SUSTENTABLE DAS8 SPA
77488652,EQD INGENIERÍA SPA,EQD INGENIERIA SPA
77516547,PACHAMAQUI PROCESOS TECNOLÓGICOS SPA,PACHAMAQUI PROCESOS TECNOLOGICOS SPA
65121447,FUNDACIÓN CHILWE,FUNDACION CHILWE
76150950,CENTRO PREMIO NOBEL MARIO MOLINA P EST EST S E...,CENTRO PREMIO NOBEL MARIO MOLINA P EST EST S E...


# Unificar datos desde ambas fuentes

In [ ]:
df_data = df_data.reset_index() # Ahora la columna index contiene el rut sin dígito verificador
df_data

,index,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio,tipo_institucion_final,Nombre normalizado
0,76794098,SERVICIOS TECNOLÓGICOS ALEJANDRO SANTIAGO PÉRE...,INSTITUCIÓN PRIVADA,76.794.098-K,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-01-2020 20:05,NaN,Empresa,SERVICIOS TECNOLOGICOS ALEJANDRO SANTIAGO PERE...
1,17349191,BERNARDITA OLIVOS URRUTIA,INSTITUCIÓN PRIVADA,17.349.191-3,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,10-06-2021 16:10,NaN,Empresa,BERNARDITA OLIVOS URRUTIA
2,72187900,JUNTA DE VECINOS Nº11 VICTOR DOMINGO SILVA,INSTITUCIÓN PRIVADA,72.187.900-3,Agrupaciones,Otros,DECRETO 5731 DE FECHA 20 DE OCTUBRE DE 1994 ot...,$ 0,$ 0,$ 0,LOS CIPRECES 1089\n ...,23-06-2005 19:01,Representante legal\nJAQUELINE PATRICIA RAIN ...,Institución Privada Sin Fines de Lucro,JUNTA DE VECINOS No11 VICTOR DOMINGO SILVA
3,76967211,RACIMOS SPA,INSTITUCIÓN PRIVADA,76.967.211-7,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,25-04-2022 18:13,NaN,Empresa,RACIMOS SPA
4,77520996,BROLLY SPA,INSTITUCIÓN PRIVADA,77.520.996-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,13-09-2022 13:35,NaN,Empresa,BROLLY SPA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,77692848,MAHARO RAPA,INSTITUCIÓN PRIVADA,77.692.848-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-09-2023 11:19,NaN,Empresa,MAHARO RAPA
157,8352508,JUSTO LORENZO GARCIA CAMPOS,INSTITUCIÓN PRIVADA,8.352.508-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,23-09-2020 15:03,NaN,Empresa,JUSTO LORENZO GARCIA CAMPOS
158,13767547,PABLO LOPEZ URBINA,INSTITUCIÓN PRIVADA,13.767.547-1,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,24-11-2020 19:08,NaN,Empresa,PABLO LOPEZ URBINA
159,4335278,IRIS AVENDAÑO BRICEÑO,INSTITUCIÓN PRIVADA,4.335.278-4,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,IDAHUILLO S/N\n ...,28-06-2007 02:03,NaN,Empresa,IRIS AVENDANO BRICENO


In [ ]:
df_data["rut_dv"] = df_data["RUT"].str.replace(".", "")
df_data["rut_dv"]

<ipython-input-60-4a9854a70239>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_data["rut_dv"] = df_data["RUT"].str.replace(".", "")


0      76794098-K
1      17349191-3
2      72187900-3
3      76967211-7
4      77520996-8
          ...    
156    77692848-8
157     8352508-8
158    13767547-1
159     4335278-4
160    77521096-6
Name: rut_dv, Length: 161, dtype: object

In [ ]:
df_data["empresa_tipo"] = None
df_data["empresa_subtipo"] = None
df_data["provincia"] = None
df_data["comuna"] = None
df_data

,index,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Sucursales,Inscripción en Registros,Directorio,tipo_institucion_final,Nombre normalizado,rut_dv,empresa_tipo,empresa_subtipo,provincia,comuna
0,76794098,SERVICIOS TECNOLÓGICOS ALEJANDRO SANTIAGO PÉRE...,INSTITUCIÓN PRIVADA,76.794.098-K,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-01-2020 20:05,NaN,Empresa,SERVICIOS TECNOLOGICOS ALEJANDRO SANTIAGO PERE...,76794098-K,None,None,None,None
1,17349191,BERNARDITA OLIVOS URRUTIA,INSTITUCIÓN PRIVADA,17.349.191-3,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,10-06-2021 16:10,NaN,Empresa,BERNARDITA OLIVOS URRUTIA,17349191-3,None,None,None,None
2,72187900,JUNTA DE VECINOS Nº11 VICTOR DOMINGO SILVA,INSTITUCIÓN PRIVADA,72.187.900-3,Agrupaciones,Otros,DECRETO 5731 DE FECHA 20 DE OCTUBRE DE 1994 ot...,$ 0,$ 0,$ 0,LOS CIPRECES 1089\n ...,23-06-2005 19:01,Representante legal\nJAQUELINE PATRICIA RAIN ...,Institución Privada Sin Fines de Lucro,JUNTA DE VECINOS No11 VICTOR DOMINGO SILVA,72187900-3,None,None,None,None
3,76967211,RACIMOS SPA,INSTITUCIÓN PRIVADA,76.967.211-7,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,25-04-2022 18:13,NaN,Empresa,RACIMOS SPA,76967211-7,None,None,None,None
4,77520996,BROLLY SPA,INSTITUCIÓN PRIVADA,77.520.996-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,13-09-2022 13:35,NaN,Empresa,BROLLY SPA,77520996-8,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,77692848,MAHARO RAPA,INSTITUCIÓN PRIVADA,77.692.848-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,14-09-2023 11:19,NaN,Empresa,MAHARO RAPA,77692848-8,None,None,None,None
157,8352508,JUSTO LORENZO GARCIA CAMPOS,INSTITUCIÓN PRIVADA,8.352.508-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,23-09-2020 15:03,NaN,Empresa,JUSTO LORENZO GARCIA CAMPOS,8352508-8,None,None,None,None
158,13767547,PABLO LOPEZ URBINA,INSTITUCIÓN PRIVADA,13.767.547-1,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,DESCONOCIDA S/N\n ...,24-11-2020 19:08,NaN,Empresa,PABLO LOPEZ URBINA,13767547-1,None,None,None,None
159,4335278,IRIS AVENDAÑO BRICEÑO,INSTITUCIÓN PRIVADA,4.335.278-4,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,IDAHUILLO S/N\n ...,28-06-2007 02:03,NaN,Empresa,IRIS AVENDANO BRICENO,4335278-4,None,None,None,None


In [ ]:
df_data_final = df_data[["index", "rut_dv", "Nombre", "empresa_tipo", "empresa_subtipo", "provincia", "comuna", "tipo_institucion_final"]].rename(columns={"index":"rut", "Nombre": "nombre"})
df_data_final

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,76794098,76794098-K,SERVICIOS TECNOLÓGICOS ALEJANDRO SANTIAGO PÉRE...,None,None,None,None,Empresa
1,17349191,17349191-3,BERNARDITA OLIVOS URRUTIA,None,None,None,None,Empresa
2,72187900,72187900-3,JUNTA DE VECINOS Nº11 VICTOR DOMINGO SILVA,None,None,None,None,Institución Privada Sin Fines de Lucro
3,76967211,76967211-7,RACIMOS SPA,None,None,None,None,Empresa
4,77520996,77520996-8,BROLLY SPA,None,None,None,None,Empresa
...,...,...,...,...,...,...,...,...
156,77692848,77692848-8,MAHARO RAPA,None,None,None,None,Empresa
157,8352508,8352508-8,JUSTO LORENZO GARCIA CAMPOS,None,None,None,None,Empresa
158,13767547,13767547-1,PABLO LOPEZ URBINA,None,None,None,None,Empresa
159,4335278,4335278-4,IRIS AVENDAÑO BRICEÑO,None,None,None,None,Empresa


In [ ]:
df_merged = pd.concat([df_final, df_data_final], ignore_index=True)
df_merged

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,77155166,77155166-1,REBORN ELECTRIC MOTORS SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,CACHAPOAL,RANCAGUA,Empresa
1,76415846,76415846-6,CONSULTORIAS Y SERVICIOS MINEROS INTEGRALES LI...,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,EL LOA,COPIAPO,Empresa
2,76055501,76055501-0,QIN TECHNOLOGY SPA.,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,LAS CONDES,Empresa
3,76940369,76940369-8,SEA GROUP SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,PROVIDENCIA,Empresa
4,76772215,76772215-K,QUEMPIN SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,VALPARAÍSO,VINA DEL MAR,Empresa
...,...,...,...,...,...,...,...,...
5073,77692848,77692848-8,MAHARO RAPA,None,None,None,None,Empresa
5074,8352508,8352508-8,JUSTO LORENZO GARCIA CAMPOS,None,None,None,None,Empresa
5075,13767547,13767547-1,PABLO LOPEZ URBINA,None,None,None,None,Empresa
5076,4335278,4335278-4,IRIS AVENDAÑO BRICEÑO,None,None,None,None,Empresa


In [ ]:
df_merged["rut_dv"] = df_merged["rut_dv"].str.lower()
df_merged["nombre"] = df_merged["nombre"].apply(unidecode)
df_merged

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,77155166,77155166-1,REBORN ELECTRIC MOTORS SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,CACHAPOAL,RANCAGUA,Empresa
1,76415846,76415846-6,CONSULTORIAS Y SERVICIOS MINEROS INTEGRALES LI...,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,EL LOA,COPIAPO,Empresa
2,76055501,76055501-0,QIN TECHNOLOGY SPA.,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,LAS CONDES,Empresa
3,76940369,76940369-8,SEA GROUP SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,SANTIAGO,PROVIDENCIA,Empresa
4,76772215,76772215-k,QUEMPIN SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,VALPARAÍSO,VINA DEL MAR,Empresa
...,...,...,...,...,...,...,...,...
5073,77692848,77692848-8,MAHARO RAPA,None,None,None,None,Empresa
5074,8352508,8352508-8,JUSTO LORENZO GARCIA CAMPOS,None,None,None,None,Empresa
5075,13767547,13767547-1,PABLO LOPEZ URBINA,None,None,None,None,Empresa
5076,4335278,4335278-4,IRIS AVENDANO BRICENO,None,None,None,None,Empresa


In [ ]:
df_merged.to_excel(os.path.join(results_dir, "data_instituciones_corfo.xlsx"), index=False)